# Encryption demo from 
https://nitratine.net/blog/post/asymmetric-encryption-and-decryption-in-python/

https://stackoverflow.com/questions/51228645/how-can-i-encrypt-with-a-rsa-private-key-in-python
https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/

## Create keys

In [9]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()

## Save private/public keys on files

In [10]:
from cryptography.hazmat.primitives import serialization
#private_key = ... # Placeholder - you generated this before

pem = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

with open('private_key.pem', 'wb') as f:
    f.write(pem)

In [11]:
#from cryptography.hazmat.primitives import serialization
#public_key = ... # Placeholder - you generated this before

pem = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

with open('public_key.pem', 'wb') as f:
    f.write(pem)

## Get private/public keys from files

In [12]:
#from cryptography.hazmat.backends import default_backend
#from cryptography.hazmat.primitives import serialization

with open("private_key.pem", "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
        backend=default_backend()
    )

In [13]:
#from cryptography.hazmat.backends import default_backend
#from cryptography.hazmat.primitives import serialization

with open("public_key.pem", "rb") as key_file:
    public_key = serialization.load_pem_public_key(
        key_file.read(),
        backend=default_backend()
    )

## Encrypting

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

message = b'encrypt me!'
#public_key = ... # Use one of the methods above to get your public key

encrypted = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

## Decrypting

In [ ]:
#from cryptography.hazmat.primitives import hashes
#from cryptography.hazmat.primitives.asymmetric import padding

#encrypted = ... # From before (could have been stored then read back here)
#private_key = ... # Use one of the methods above to get your public key (matches the public_key)

original_message = private_key.decrypt(
    encrypted,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

## Encrypting and decrypting files

In [ ]:
f = open('test.txt', 'rb')
message = f.read()
f.close()

In [ ]:
encrypted = 'data from encryption'
f = open('test.encrypted', 'wb')
f.write(encrypted)
f.close()

# Test

In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

In [2]:
"""
Creates the keys in files
n is an int to use as id for the keys
"""
def create_keys(n):
    #Create the keys
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    
    #Store the keys
    pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    with open('private_key_'+str(n)+'.pem', 'wb') as f:
        f.write(pem)
        
    pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    with open('public_key_'+str(n)+'.pem', 'wb') as f:
        f.write(pem)

In [3]:
"""
Function to retrieve the keys from the files using int n to identify them
returns tuple of private and public keys
"""
def get_keys(n):
    with open('private_key_'+str(n)+'.pem', "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )
    with open('public_key_'+str(n)+'.pem', "rb") as key_file:
        public_key = serialization.load_pem_public_key(
            key_file.read(),
            backend=default_backend()
        )
    return (private_key,public_key)

In [4]:
"""
Sign a file with sender private key 
"""
def sign_file(file,sender):
    f = open(file, 'rb')
    message = f.read()
    f.close()
    #sender private key
    encrypted = get_keys(sender)[0].sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    f = open('signature.bin', 'wb')
    f.write(encrypted)
    f.close()

In [5]:
"""
Verify a singature with sender public key
"""
def verify_signature(file,sender):
    f = open('signature.bin', 'rb')
    signature = f.read()
    f.close()
    
    f = open(file, 'rb')
    original_message = f.read()
    f.close()
    try:
        original_message2 = get_keys(sender)[1].verify(
            signature,
            original_message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        pass
    except:
        print("Invalid signature!")
        pass
    else:
        print("Validated signature!")
        pass

In [6]:
create_keys(1)
create_keys(2)

In [7]:
print(get_keys(1))
print(get_keys(2))

(<cryptography.hazmat.backends.openssl.rsa._RSAPrivateKey object at 0x0000029A922DDEB0>, <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x0000029A922DDE20>)
(<cryptography.hazmat.backends.openssl.rsa._RSAPrivateKey object at 0x0000029A922DDE80>, <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x0000029A922DDCD0>)


In [11]:
sign_file('Document.txt',2)

In [12]:
verify_signature('Document.txt',2)

Validated signature!
